# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

With the help of python notebooks provided in 1st and 2nd project of this nanodegree program ('Optimizing a pipeline in Azure' and 'Operationalizing machine learning'), I have imported following basic dependencies required to complete this project. Any other specific dependecy will be imported as we proceed further in this project.

In [2]:
import logging
import os
import csv

from matplotlib import pyplot as pyplot
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# CheckIng core SDK version number
print("SDK Version:", azureml.core.VERSION)

SDK Version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

I will be using the 'Heart Failure Clinical Data' which consists of 12 features ( age, anaemia, creatinine_phosphokinase, diabetes, ejection_fraction, high_blood_pressure, platelets, serum_creatinine, serum_sodium, sex, smoking, time ) which can be used to predict mortality by heart failure. There are total of 299 input rows in the dataset with 0 null entries.

The 12 features are as follows:

(1) age

(2) anaemia i.e. decrease of red blood cells or hemoglobin (boolean)

(3) creatining_phosphokinase i.e. level of the CPK enzyme in the blood (mcg/L)

(4) diabetes i.e. if the patient has diabetes or not (boolean)

(5) ejection_fraction i.e. percentage of blood leaving the heart at each contraction (percentage)

(6) high_blood_pressure i.e. if the patient has hypertension (boolean)

(7) platelets i.e. platelets in the blood (kiloplatelets/mL)

(8) serum_creatinine i.e. level of serum creatinine in the blood (mg/dL)

(9) serum_sodium i.e. level of serum sodium in the blood (mEq/L)

(10) sex i.e. woman or man (binary)

(11) smoking i.e. if the patient smokes or not (boolean)

(12) time i.e. follow-up period (days)


We will be predicting the following output:

DEATH_EVENT i.e if the patient deceased during the follow-up period (boolean)

A machine learning classification model on this dataset will be helpful for early detection of people with cardiovascular disease or those who are at high risk of cardiovascular disease. 

SOURCE : https://www.kaggle.com/andrewmvd/heart-failure-clinical-data

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

I have already registered the dataset after downloading it from kaggle. So, I will be using the name and description that I saved the dataset with, to import it in my experiment.

In [3]:
# creating an automl experiment in our workspace

# initializing a workspace
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

# choosing a name for experiment
experiment_name = 'capstone-automl-experiment'
project_folder = './pipeline-project'

# creating the experiment
experiment = Experiment(ws, experiment_name)
experiment.start_logging()
experiment

quick-starts-ws-134993
aml-quickstarts-134993
southcentralus
b968fb36-f06a-4c76-a15f-afab68ae7667


Name,Workspace,Report Page,Docs Page
capstone-automl-experiment,quick-starts-ws-134993,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
# creating an AMLCompute cluster for running the experiment

# importing required dependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choosing a name for our CPU cluster
amlcompute_cluster_name = "aml-auto"

# Verifying that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
compute_target.get_status()

Creating
Succeeded.................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [5]:
# importing required dependency
from azureml.data.dataset_factory import TabularDatasetFactory

# importing dataset using url
dataset = TabularDatasetFactory.from_delimited_files("https://raw.githubusercontent.com/ujjwalbb30/nd00333-capstone/ujjwalbb30-patch-1/heart_failure_clinical_records_dataset.csv")

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

for automl settings, I will be using the following parameters:

(1) experiment_timeout_minutes : It is the amount of time that the experiment will run upto. I will input it as 30 minutes which means the the experiment will exit after 30 minutes ( if it doesn't find the best run within 30 minutes and exit on its own ) and will give out the best result found during that time.

(2) max_concurrent_iterations : It is the maximum number of iterations to be executed in parallel. I will input it as '5' iterations. 5 concurrent iterations will help in quickly executing the tasks of experiment and it will also not load the compute target too much for computation.

(3) primary_metric : This is the metric that will be optimized by Automated Machine Learning for model selection. I will use 'AUC_weighted' as 'primary_metric' parameter. AUC means the area under the Receiver Operating Characteristic Curve which plots the relationship between true positive rate and false positive rate. Since our dataset doesn't have high class imbalance, we can use ROC method for judging the performance of a model. I will use AUC_weighted in order to mitigate the effects of whatever little imbalance is there in the dataset. AUC_weighted is the arithmetic mean of the score for each class, weighted by the number of true instances in each class.

for automl configuration, I will be using the following parameters:

(1) compute_target : It is the compute target on which we will run our Azure Machine Learning experiment. Since I have created a compute target named as 'compute_target' for this purpose, I will input it as the 'compute_target' parameter.

(2) task : I want to make a classification model that can predict whether the patient is at a high risk of cardiovascular disease or not. Hence, I will input 'classification' as 'task' parameter.

(3) training_data : It is the training dataset to be used for the experiment. I will use 'dataset' (the registered dataset imported above for running this experiment) as 'training_data' parameter. importing training dataset means the output columns will be included and its name will be entered in 'label_column_name'.

(4) label_column_name : It is the name of the output column present in the training dataset. I will enter 'DEATH_EVENT' as 'label_column_name' parameter.

(5) path : This is the full path to the Azure Machine learning project folder. Hence, I will input './pipeline-project' as 'path' parameter.

(6) enable_early_stopping : we can choose to terminate the experiment if the score stops improving in the short term. I will enter 'True' as 'enable_early_stopping' parameter.

(7) featurization : It is the option to featurize the dataset i.e. whether we want the Azure to do it automatically or we want to turn it off or we want some customized featurization step. I will input 'auto' in the 'featurization' parameter as I want Azure to featurize the dataset automatically.

(8) debug_log : it is the log file in which debug information is written. I am entering 'automl_errors.log' as 'debug_log' parameter.

(9) n_cross_validations : It is the number of cross validations performed. I will input it as '5' since the input rows is way lower than 1000 and 5 cross validations will not be very computation expensive.

In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes" : 30,
    "max_concurrent_iterations" : 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                            task="classification",
                            training_data=dataset,
                            label_column_name="DEATH_EVENT",
                            path='./pipeline-project',
                            enable_early_stopping=True,
                            featurization='auto',
                            enable_onnx_compatible_models=True,
                            debug_log="automl_errors.log",
                            **automl_settings
)

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(config = automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on aml-auto with default configuration
Running on remote compute: aml-auto
Parent Run ID: AutoML_19bb6a27-9e6e-4ada-90a3-44f64b282aa5

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed,

In [9]:
# waiting for completion of remote_run while showing its output
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTI

{'runId': 'AutoML_19bb6a27-9e6e-4ada-90a3-44f64b282aa5',
 'target': 'aml-auto',
 'status': 'Completed',
 'startTimeUtc': '2021-01-17T06:59:53.73799Z',
 'endTimeUtc': '2021-01-17T07:26:55.949278Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'aml-auto',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-automl-experiment","subscription_id":"b968fb36-f06a-4c76-a15f-afab68ae7667","resource_group":"aml-quickstarts-134993","workspace_name":"quick-starts-ws-134993","region":"southcentralus","compute_target":"aml-auto","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_c

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
# importing required dependencies
from azureml.widgets import RunDetails

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
# Retrieve best model from AutoML Run

# importing required dependencies
import joblib

best_automl_run, best_automl_fitted_model = remote_run.get_output()
print(best_automl_run)
print(best_automl_fitted_model)

Run(Experiment: capstone-automl-experiment,
Id: AutoML_19bb6a27-9e6e-4ada-90a3-44f64b282aa5_52,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_leaf=0.01,
                                                                                                  min_samples_split=0.1505263157894

In [12]:
#TODO: Save the best model
joblib.dump(best_automl_fitted_model,'best_automl_model.pkl')

['best_automl_model.pkl']

In [13]:
# getting the details of the best model produced by automl
best_automl_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":3,"CurrentNodeCount":4}',
 'ensembled_iterations': '[33, 23, 46, 48, 12, 20, 7]',
 'ensembled_algorithms': "['RandomForest', 'RandomForest', 'GradientBoosting', 'GradientBoosting', 'RandomForest', 'RandomForest', 'ExtremeRandomTrees']",
 'ensemble_weights': '[0.16666666666666666, 0.25, 0.08333333333333333, 0.08333333333333333, 0.16666666666666666, 0.16666666666666666, 0.08333333333333333]',
 'best_individual_pipeline_score': '0.9137252395921733',
 'best_individual_iteration': '33',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}

In [14]:
from pprint import pprint

def print_model(model,prefix=""):
    for step in model.steps:
        print(prefix+step[0])
        if hasattr(step[1],'estimators') and hasattr(step[1],'weights'):
            pprint({'estimators':list(e[0] for e in step[1].estimators), 'weights':step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1],estimator[0]+'-')
        else:
            pprint(step[1].get_params())
            print()

print_model(best_automl_fitted_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['33', '23', '46', '48', '12', '20', '7'],
 'weights': [0.16666666666666666,
             0.25,
             0.08333333333333333,
             0.08333333333333333,
             0.16666666666666666,
             0.16666666666666666,
             0.08333333333333333]}

33-minmaxscaler
{'copy': True, 'feature_range': (0, 1)}

33-randomforestclassifier
{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': None,
 'max_features': 0.3,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 0.01,
 'min_

In [15]:
# retrieving the best model as ONNX model
best_auto_run, best_onnx_model = remote_run.get_output(return_onnx_model=True)

In [16]:
# importing required dependencies
from azureml.automl.runtime.onnx_convert import OnnxConverter

# saving the best model as onnx_model
onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(best_onnx_model, onnx_fl_path)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
# registering the best automl model

description = 'heart failure predictions'
tags = None

model = remote_run.register_model(description = description, tags = tags)

print(remote_run.model_id)

# importing required dependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

# loading a curated environment from workspace

env = Environment.get(ws, "AzureML-AutoML")

# specifying scikit-learn as dependency
for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

# creating an inference config
inference_config = InferenceConfig(entry_script='entry_script.py', environment=env)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, enable_app_insights = True)

# naming the service to be deployed
aci_service_name = 'automl-heart-failure-predictions'
print(aci_service_name)

# deploying the model
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)

AutoML19bb6a27952
automl-heart-failure-predictions
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [22]:
# importing required dependencies
from numpy import array
import json
import requests

# creating a test sample
data = {"data": [{"age":60.000000,"anaemia":0.000000,"creatinine_phosphokinase":250.000000,"diabetes":0.000000,"ejection_fraction":38.000000,"high_blood_pressure":0.000000,"platelets":262000.000000,"serum_creatinine":1.10000,"serum_sodium":137.000000,"sex":1.000000,"smoking":0.00000,"time":115.000000}]}
td = json.dumps(data)
headers = {'Content-Type': 'application/json'}

# sending request to test the deployed webservice
resp = requests.post(aci_service.scoring_uri, td, headers=headers)
print(resp.json())
y_pred = (json.loads(resp.text))
print(y_pred)

[0]
[0]


TODO: In the cell below, print the logs of the web service and delete the service

In [24]:
# printing the logs of deployed web service
dep_logs = aci_service.get_logs()
for l in dep_logs.split('\n'):
    print(l)

2021-01-17T07:55:38,080444900+00:00 - iot-server/run 
2021-01-17T07:55:38,085328100+00:00 - gunicorn/run 
2021-01-17T07:55:38,084755800+00:00 - rsyslog/run 
2021-01-17T07:55:38,121335800+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_265db83b0c6014ce472c5de2f0b97e04/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [25]:
# deleting a web service
aci_service.delete()

In [26]:
# delete compute cluster
compute_target.delete()